In [47]:
import os
import pickle
import numpy as np
from glob import glob
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

In [48]:
class DataHandler:
    """Handles loading and processing MIDI data for training."""
    
    def __init__(self, midi_folder='Data', num_files=20):
        self.midi_folder = midi_folder
        self.num_files = num_files

    def load_midi_files(self):
        """Loads MIDI files from the specified folder up to a specified maximum number."""
        path = os.path.join(self.midi_folder, '*.mid')
        files = glob(path)
        return files[:self.num_files]

    def get_notes(self):
        """Extracts notes and chords from MIDI files."""
        notes = []
        for file in self.load_midi_files():
            midi = converter.parse(file)
            parts = instrument.partitionByInstrument(midi)
            notes_to_parse = parts.parts[0].recurse() if parts else midi.flat.notes
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
        return notes

    def prepare_sequences(self, notes, sequence_length=100):
        """Prepares training sequences for the model."""
        pitchnames = sorted(set(notes))
        note_to_int = {note: number for number, note in enumerate(pitchnames)}
        network_input = []
        network_output = []
        
        for i in range(len(notes) - sequence_length):
            sequence_in = notes[i: i + sequence_length]
            sequence_out = notes[i + sequence_length]
            network_input.append([note_to_int[char] for char in sequence_in])
            network_output.append(note_to_int[sequence_out])
        
        n_patterns = len(network_input)
        network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
        network_input = network_input / float(len(pitchnames))
        network_output = to_categorical(network_output, num_classes=len(pitchnames))
        
        return network_input, network_output


In [49]:
class MusicModel:
    """Builds and trains a neural network for music generation."""
    
    def __init__(self, model_path='Model', lstm_units=(128, 128), dropout_rates=(0.2, 0.3), dense_units=256):
        self.model_path = model_path
        self.lstm_units = lstm_units
        self.dropout_rates = dropout_rates
        self.dense_units = dense_units
        if not os.path.exists(self.model_path):
            os.makedirs(self.model_path)

    def create_network(self, input_shape, n_vocab):
        """Creates the LSTM network model."""
        model = Sequential()
        model.add(Input(shape=input_shape))  # Explicit input layer
        model.add(LSTM(self.lstm_units[0], return_sequences=True))
        model.add(Dropout(self.dropout_rates[0]))
        model.add(LSTM(self.lstm_units[1]))
        model.add(Dropout(self.dropout_rates[1]))
        model.add(Dense(self.dense_units))
        model.add(Dense(n_vocab, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam')
        return model

    def train(self, network_input, network_output, epochs=50, batch_size=64):
        """Trains the model and saves the best performing model."""
        model = self.create_network((network_input.shape[1], network_input.shape[2]), network_output.shape[1])
        filepath = os.path.join(self.model_path, f'weights.best.music.keras')
        checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
        model.fit(network_input, network_output, epochs=epochs, batch_size=batch_size, callbacks=[checkpoint])


In [50]:
class MusicGenerator:
    """Generates music using the trained model."""
    
    def __init__(self, model_path='Model', output_path='Output/output.mid'):
        self.model_path = model_path
        self.output_path = output_path

    def load_model(self):
        """"Loads the trained model."""
        return load_model(os.path.join(self.model_path, 'weights.best.music.keras'))

    def generate_music(self, model, notes, n_vocab, length=500, temperature=1.0):
        """Generates a sequence of music notes."""
        int_to_note = {number: note for number, note in enumerate(sorted(set(notes)))}
        pattern = np.random.randint(0, n_vocab, size=(100,)).tolist()
        prediction_output = []

        for note_index in range(length):
            prediction_input = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
            prediction = model.predict(prediction_input, verbose=0).astype('float64')
            prediction = np.log(prediction + 1e-7) / temperature  # Smoothing
            exp_preds = np.exp(prediction)
            prediction = exp_preds / np.sum(exp_preds)
            index = np.random.choice(range(n_vocab), p=prediction[0])
            result = int_to_note[index]
            prediction_output.append(result)
            pattern.append(index)
            pattern = pattern[1:]

        return prediction_output
    
    def create_midi(self, prediction_output):
        """Converts the generated note sequence into a MIDI file."""
        offset = 0
        output_notes = []

        for pattern in prediction_output:
            # Handling chords
            if ('.' in pattern) or pattern.isdigit():
                notes_in_chord = pattern.split('.')
                notes = [note.Note(int(n)) for n in notes_in_chord]
                new_chord = chord.Chord(notes)
                new_chord.offset = offset
                output_notes.append(new_chord)
            else: # Handling single notes
                new_note = note.Note(pattern)
                new_note.offset = offset
                output_notes.append(new_note)

            offset += 0.5

        midi_stream = stream.Stream(output_notes)
        midi_stream.write('midi', fp=self.output_path)


In [51]:
class MusicPlayer:
    """Plays MIDI music files."""
    def __init__(self, midi_file_path):
        self.midi_file_path = midi_file_path

    def play_midi(self):
        """Uses music21 to play a MIDI file."""
        from music21 import midi
        midi_stream = converter.parse(self.midi_file_path)
        sp = midi.realtime.StreamPlayer(midi_stream)
        sp.play()


In [56]:
"""Implementing the Neural Network and Training it"""
data_handler = DataHandler()

notes = data_handler.get_notes()
if len(notes) > 100:
    n_vocab = len(set(notes))
    network_input, network_output = data_handler.prepare_sequences(notes)

    model_trainer = MusicModel()

    model_trainer.train(network_input, network_output, epochs=100)

else:
    print("Not enough notes to proceed with training and music generation.")


Epoch 1/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - loss: 4.0917
Epoch 1: loss improved from inf to 3.95932, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 23s 129ms/step - loss: 4.0909
Epoch 2/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 3.8479
Epoch 2: loss improved from 3.95932 to 3.82737, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 23s 142ms/step - loss: 3.8477
Epoch 3/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 3.7826
Epoch 3: loss improved from 3.82737 to 3.76781, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 22s 132ms/step - loss: 3.7825
Epoch 4/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 3.7391
Epoch 4: loss improved from 3.76781 to 3.73371, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 22s 131ms/step - loss: 3.7391
Epoch 5/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 3.7206
Epoch 5: loss improved from 3.73371 to

Epoch 29/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 2.8094
Epoch 29: loss improved from 2.85726 to 2.80825, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 21s 130ms/step - loss: 2.8094
Epoch 30/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 2.7621
Epoch 30: loss improved from 2.80825 to 2.75428, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 22s 133ms/step - loss: 2.7620
Epoch 31/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 2.6968
Epoch 31: loss improved from 2.75428 to 2.70081, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 22s 134ms/step - loss: 2.6968
Epoch 32/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 2.6303
Epoch 32: loss improved from 2.70081 to 2.64997, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 21s 130ms/step - loss: 2.6304
Epoch 33/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - loss: 2.6353
Epoch 33: loss improved f

165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - loss: 2.0246
Epoch 57: loss improved from 2.08480 to 2.06840, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 25s 150ms/step - loss: 2.0249
Epoch 58/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - loss: 2.0152
Epoch 58: loss improved from 2.06840 to 2.04759, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 24s 148ms/step - loss: 2.0154
Epoch 59/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - loss: 1.9952
Epoch 59: loss improved from 2.04759 to 2.02582, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 25s 151ms/step - loss: 1.9954
Epoch 60/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - loss: 2.0112
Epoch 60: loss improved from 2.02582 to 2.02084, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 26s 156ms/step - loss: 2.0113
Epoch 61/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - loss: 1.9788
Epoch 61: loss did not improve from 2.

165/165 ━━━━━━━━━━━━━━━━━━━━ 21s 128ms/step - loss: 1.6433
Epoch 87/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - loss: 1.6263
Epoch 87: loss improved from 1.67079 to 1.66295, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 21s 129ms/step - loss: 1.6265
Epoch 88/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 1.6099
Epoch 88: loss improved from 1.66295 to 1.65006, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 22s 133ms/step - loss: 1.6101
Epoch 89/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - loss: 1.6132
Epoch 89: loss improved from 1.65006 to 1.64754, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 23s 139ms/step - loss: 1.6134
Epoch 90/100
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - loss: 1.5891
Epoch 90: loss improved from 1.64754 to 1.62071, saving model to Model/weights.best.music.keras
165/165 ━━━━━━━━━━━━━━━━━━━━ 22s 135ms/step - loss: 1.5893
Epoch 91/100
165/165 ━━━━━━━━━━━━━━━━

In [57]:
notes = data_handler.get_notes()
n_vocab = len(set(notes))

In [54]:
music_gen = MusicGenerator()
model = music_gen.load_model()
prediction_output = music_gen.generate_music(model=model, notes=notes, n_vocab=n_vocab)
music_gen.create_midi(prediction_output)
print("Output Created!")

Output Created!


In [55]:
player = MusicPlayer('Output/output.mid')

# player.play_midi()